# Secure Data Disclosure: Server side

This notebook showcases how data owner could set up the server, add make their data available to certain user. It explains the different steps necessary.

## Create a docker volume
The first step is to create a docker volume for mongodb. Docker volumes are persistent storage spaces that are managed by docker and can be mounted in containers. To create the volume use `docker volume create mongodata`. This must be done only once, and we use bind mounts for the server, so no need to create volumes for that.

In [2]:
!docker volume create mongodata

mongodata


## Start server
The second step is to start the server. Therefore the config file `configs/example_config.yaml` has to be adapted. The data owner must make sure to set the develop mode to False, specify the database type and ports. For this notebook, we will keep the default and use a mongodb on port 27017. Note: Keep in mind that if the configuration file is modified then the `docker-compose` has to be modified accordingly. This is out of scope for this notebook.

Startup the service by running `docker compose up` in a terminal.

In [6]:
#!docker compose down

## Access the database

In [36]:
# MongoDB
MONGODB_CONTAINER_NAME = "mongodb"
MONGODB_PORT = "27017"
DATABASE_NAME = "user_database"

In [37]:
from mongodb_admin import MongoDB_Admin

mongo_admin = MongoDB_Admin(
    f"mongodb://{MONGODB_CONTAINER_NAME}:{MONGODB_PORT}/"
)

In [46]:
db = mongo_admin.db[DATABASE_NAME]
db

Collection(Database(MongoClient(host=['mongodb:27017'], document_class=dict, tz_aware=False, connect=True), 'user_database'), 'user_database')

For now the database is empty. We will create a collection named `users` and fill it in the next steps.

In [53]:
db.users.count_documents({"user_name": "Alice"})

ServerSelectionTimeoutError: mongodb:27017: [Errno -2] Name or service not known, Timeout: 30s, Topology Description: <TopologyDescription id: 649c61abfd4944b39ce72842, topology_type: Unknown, servers: [<ServerDescription ('mongodb', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('mongodb:27017: [Errno -2] Name or service not known')>]>

In [42]:
d = dict((db, [collection for collection in client[db].collection_names()]) for db in mongo_admin.database_names())
print(json.dumps(d))

AttributeError: 'MongoDB_Admin' object has no attribute 'database_names'

## Visualise all options
You can visualise all the options offered by the database by running the command `python src/mongodb_admin.py --help`. We will go through through each of them in the rest of the notebook.

In [24]:
!python src/mongodb_admin.py --help

usage: MongoDB administration script for the SDD POC Server
       [-h]
       {add_user,del_user,add_dataset_to_user,del_dataset_to_user,set_budget_field,set_may_query,add_metadata,del_metadata,drop_collection,create_ex_users}
       ...

options:
  -h, --help            show this help message and exit

subcommands:
  {add_user,del_user,add_dataset_to_user,del_dataset_to_user,set_budget_field,set_may_query,add_metadata,del_metadata,drop_collection,create_ex_users}
                        database administration operations
    add_user            add user to users collection
    del_user            delete user from users collection
    add_dataset_to_user
                        add dataset with initialized budget values for a user
                        in users collection
    del_dataset_to_user
                        delete dataset for user in users collection
    set_budget_field    set budget field to given value for given user and
                        dataset
    set_may_que

## Add dataset
The super confidential dataset that we want to add is the penguin dataset from.

In [10]:
DATASET = "PENGUIN" # TODO because nothing yet

## Add user
Let's add few users. For now, you can just give their name.

In [ ]:
!python src/mongodb_admin.py add_user --user "Alice"

In [54]:
!python src/mongodb_admin.py add_user --user "Dr. Antartica"

Traceback (most recent call last):
  File "/home/pauline/poc/sdd-poc-server/src/mongodb_admin.py", line 302, in <module>
    args.func(args)
  File "/home/pauline/poc/sdd-poc-server/src/mongodb_admin.py", line 32, in add_user
    if self.db.users.count_documents({"user_name": args.user}) > 0:
  File "/home/pauline/anaconda3/lib/python3.10/site-packages/pymongo/collection.py", line 1835, in count_documents
    return self._retryable_non_cursor_read(_cmd, session)
  File "/home/pauline/anaconda3/lib/python3.10/site-packages/pymongo/collection.py", line 1840, in _retryable_non_cursor_read
    with client._tmp_session(session) as s:
  File "/home/pauline/anaconda3/lib/python3.10/contextlib.py", line 135, in __enter__
    return next(self.gen)
  File "/home/pauline/anaconda3/lib/python3.10/site-packages/pymongo/mongo_client.py", line 1729, in _tmp_session
    s = self._ensure_session(session)
  File "/home/pauline/anaconda3/lib/python3.10/site-packages/pymongo/mongo_client.py", line 1712, i

In [ ]:
Nice, we have now these users are added

## Remove user

## Change budget

## Stop the server
To tear down the service, use `docker compose down`. This will also delete all the containers but the volume will stay in place.

In [1]:
!docker compose down